In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import datetime
import json
import os
import pickle
from pprint import pprint
import sys

# TF logging control
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

project_root = os.path.abspath('..')
sys.path.append(project_root)

from sleeprnn.data import utils
from sleeprnn.detection.feeder_dataset import FeederDataset
from sleeprnn.detection.predicted_dataset import PredictedDataset
from sleeprnn.nn.models import WaveletBLSTM
from sleeprnn.helpers.reader import load_dataset
from sleeprnn.common import constants
from sleeprnn.common import checks
from sleeprnn.common import pkeys

RESULTS_PATH = os.path.join(project_root, 'results')

In [5]:
n2_subsampling_factors_dict = {
    constants.MODA_SS_NAME: [12.5, 25.0, 50.0, 100.0],  # [15.0, 30.0, 60.1, 100.0],
    constants.MASS_SS_NAME: [6.2, 12.5, 25.0, 50.0, 100.0],
    constants.INTA_SS_NAME: [11.4, 22.8, 45.5, 100.0],
    constants.CAP_SS_NAME: [5, 10, 20, 40, 80, 100],
}

datasets_list = [
    dict(dataset_name=constants.MASS_SS_NAME, which_expert=1),
    # dict(dataset_name=constants.MODA_SS_NAME, which_expert=1),
    # dict(dataset_name=constants.INTA_SS_NAME, which_expert=1),
    # dict(dataset_name=constants.CAP_SS_NAME, which_expert=1),
]
ds = datasets_list[0]

dataset_name = ds['dataset_name']
which_expert = ds['which_expert']
dataset = load_dataset(dataset_name, verbose=False)
subject_ids =  dataset.all_ids

In [6]:
print("Dataset:", dataset_name)
print("ids", subject_ids)
for train_size in n2_subsampling_factors_dict[dataset_name]:
    data_feed = FeederDataset(
        dataset, subject_ids, constants.N2_RECORD, which_expert=which_expert,
        n2_subsampling_factor=(train_size / 100))
    all_n2_pages = data_feed.get_pages(pages_subset='n2')
    n2_sum = np.concatenate(all_n2_pages).size
    
    all_events = data_feed.get_stamps(pages_subset='n2')
    events_sum = np.concatenate(all_events, axis=0).shape[0]
    
    print("\nSize %s, number of events retrieved %d" % (train_size, events_sum))
    print("Size %s, number of pages retrieved %d" % (train_size, n2_sum))
    print("pages", [p.size for p in all_n2_pages])
    print("events", [p.shape[0] for p in all_events])

Dataset: mass_ss
ids [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 17, 18, 19]

Size 6.2, number of events retrieved 11
Size 6.2, number of pages retrieved 677
pages [47, 57, 31, 40, 42, 44, 46, 44, 40, 43, 51, 46, 50, 46, 50]
events [0, 1, 0, 0, 0, 5, 1, 0, 1, 0, 0, 2, 0, 1, 0]

Size 12.5, number of events retrieved 107
Size 12.5, number of pages retrieved 1357
pages [94, 114, 62, 80, 85, 87, 93, 87, 80, 87, 103, 92, 100, 92, 101]
events [13, 19, 1, 7, 3, 20, 11, 0, 3, 1, 0, 13, 2, 10, 4]

Size 25.0, number of events retrieved 633
Size 25.0, number of pages retrieved 2706
pages [187, 228, 124, 159, 169, 174, 185, 174, 160, 173, 205, 183, 200, 184, 201]
events [48, 94, 5, 22, 7, 99, 45, 38, 36, 27, 32, 62, 18, 70, 30]

Size 50.0, number of events retrieved 2314
Size 50.0, number of pages retrieved 5404
pages [373, 456, 248, 317, 338, 347, 370, 348, 319, 345, 409, 365, 399, 368, 402]
events [206, 287, 31, 91, 48, 286, 175, 152, 103, 151, 139, 147, 82, 333, 83]

Size 100.0, number of events 